In [21]:
from dotenv import load_dotenv

load_dotenv()

False

In [22]:
from langchain_community.document_loaders import SeleniumURLLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# from langchain_upstage import ChatUpstage
from langchain_openai import ChatOpenAI

In [30]:
# 1st method
import requests
from bs4 import BeautifulSoup
from langchain.docstore.document import Document
from typing import List
import time

def simple_scrape(urls: List[str]) -> List[Document]:
    # Set headers to mimic a browser
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Referer": "https://www.google.com/"
    }
    
    documents = []
    
    for url in urls:
        try:
            # Add short delay between requests
            time.sleep(1)
            
            # Send request
            response = requests.get(url, headers=headers, timeout=10)
            
            if response.status_code == 200:
                # Parse HTML and extract text
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # Remove script and style tags
                for tag in soup(["script", "style"]):
                    tag.decompose()
                
                # Extract text
                text = soup.get_text(separator='\n')
                title = soup.title.string if soup.title else url
                
                # Create Document
                doc = Document(page_content=text, metadata={"source": url, "title": title})
                documents.append(doc)
                print(f"Loaded: {url}")
            else:
                print(f"Failed (status code {response.status_code}): {url}")
                
        except Exception as e:
            print(f"Error: {url}, {str(e)}")
    
    return documents

# Usage example
urls = [
    "https://wikibook.co.kr/vector-search/",
    "https://wikibook.co.kr/llm-apps/",
    "https://wikibook.co.kr/copilot/",
    "https://wikibook.co.kr/my-chatgpt/",
    "https://wikibook.co.kr/langchain/"
]

docs = simple_scrape(urls)
print(f"Total Loaded Count: {len(docs)}")

Loaded : https://wikibook.co.kr/vector-search/
Loaded : https://wikibook.co.kr/llm-apps/
Loaded : https://wikibook.co.kr/copilot/
Loaded : https://wikibook.co.kr/my-chatgpt/
Loaded : https://wikibook.co.kr/langchain/
Total Loaded Count: 5


In [27]:
# 2nd method
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (optional)
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
# Change to a common browser User-Agent string
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                            "AppleWebKit/537.36 (KHTML, like Gecko) "
                            "Chrome/115.0.0.0 Safari/537.36")

# Initialize the Chrome driver with the specified options
driver = webdriver.Chrome(options=chrome_options)

urls = [
    "https://wikibook.co.kr/vector-search/",
    "https://wikibook.co.kr/llm-apps/",
    "https://wikibook.co.kr/copilot/",
    "https://wikibook.co.kr/my-chatgpt/",
    "https://wikibook.co.kr/langchain/"
]

docs = []
for url in urls:
    try:
        driver.get(url)
        time.sleep(2)  # Wait for the page to fully load
        docs.append(driver.page_source)
        print(f"Loaded: {url}")
    except Exception as e:
        print(f"Error fetching or processing {url}, exception: {e}")

driver.quit()

Loaded: https://wikibook.co.kr/vector-search/
Loaded: https://wikibook.co.kr/llm-apps/
Loaded: https://wikibook.co.kr/copilot/
Loaded: https://wikibook.co.kr/my-chatgpt/
Loaded: https://wikibook.co.kr/langchain/


In [35]:
docs

[Document(metadata={'source': 'https://wikibook.co.kr/vector-search/', 'title': '엘라스틱서치를 활용한 벡터 검색 실무 가이드 | 위키북스'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n엘라스틱서치를 활용한 벡터 검색 실무 가이드 | 위키북스\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nToggle navigation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAbout \n\n\n\n\n회사 소개\n\n\n찾아오시는 길\n\n\n\n\n\n\n\n\n도서 시리즈 \n\n\n\n\n헬로! 인공지능\n\n\n\n\n메타버스\n\n\n\n\n콘텐츠 크리에이션\n\n\n\n\n그래픽 & 아트\n\n\n\n\nwith AI\n\n\n\n\n데이터 자격검정\n\n\n\n\n생성형 AI 프로그래밍\n\n\n\n\n위키미디어\n\n\n\n\nbe 개발자\n\n\n\n\nIT Leaders\n\n\n\n\nTECH@NAVER\n\n\n\n\nUX 디자인\n\n\n\n\n게임 개발\n\n\n\n\n네이버 스타트북\n\n\n\n\n맥 & 라이프\n\n\n\n\n시스템 & 네트워킹\n\n\n\n\n어비의 실무노트\n\n\n\n\n오픈소스 & 웹\n\n\n\n\n웹동네\n\n\n\n\n임베디드 & 모바일\n\n\n\n\n프로그래밍 & 프랙티스\

In [33]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following in Korean:\n\n{context}")]
)
output_parser = StrOutputParser()

# model = ChatUpstage()
model = ChatOpenAI(model="gpt-4o-mini")

chain = (
    prompt
    | model
    | output_parser
)

In [34]:
for doc in docs:
    print(f"URL: {doc.metadata['source']}")
    print(f"Title: {doc.metadata['title']}")
    summary = chain.invoke({"context": doc.page_content})
    print(f"Summary: {summary}\n\n")

URL: https://wikibook.co.kr/vector-search/
Title: 엘라스틱서치를 활용한 벡터 검색 실무 가이드 | 위키북스
Summary: "엘라스틱서치를 활용한 벡터 검색 실무 가이드"는 자연어 처리와 생성형 AI, RAG를 위한 대규모 벡터 데이터베이스 구축을 다룬 책입니다. 이 책은 벡터 검색과 사이버 보안, 가관측성 최적화 방법을 제공하며, 기초부터 고급 애플리케이션, 성능 튜닝 및 특수 사용 사례를 포괄적으로 설명합니다. 저자는 벡터를 활용한 이미지 검색, 개인 식별 정보(PII) 처리, 로그 분석과 사이버 보안 강화를 위한 다양한 기법을 소개합니다. 독자는 이 책을 통해 엘라스틱을 활용하여 벡터 검색을 효과적으로 구현하고 최적화하는 데 필요한 기술을 익힐 수 있습니다.


URL: https://wikibook.co.kr/llm-apps/
Title: 챗GPT와 랭체인을 활용한 LLM 기반 AI 앱 개발 | 위키북스
Summary: 이 책은 챗GPT와 랭체인 프레임워크를 활용하여 대규모 언어 모델(LLM) 기반의 AI 애플리케이션을 개발하는 방법을 단계별로 소개합니다. OpenAI API와 랭체인의 기본 개념부터 시작하여, 실제 웹 애플리케이션과 슬랙 앱 구현에 이르기까지의 과정을 다룹니다. 또한 최신 기술인 벡터 데이터베이스, 서버리스 아키텍처 활용까지 포함되며, 실무에 필요한 보안 및 개인정보보호 관련 노하우도 배울 수 있습니다. 이 책을 통해 독자들은 LLM의 잠재력을 현실에 적용하는 개발자로서의 역량을 넓힐 수 있을 것입니다.


URL: https://wikibook.co.kr/copilot/
Title: AI 코딩 어시스턴트 깃허브 코파일럿 제대로 활용하기 | 위키북스
Summary: "AI 코딩 어시스턴트 깃허브 코파일럿 제대로 활용하기"는 초보자도 쉽게 따라 할 수 있도록 구성된 파이썬 프로그래밍 입문서입니다. 이 책은 AI 도구인 깃허브 코파일럿과 챗GPT를 활용하여 코드를 작성하고 개선하는 방법을 제시합니다.